<a href="https://colab.research.google.com/github/ttb-folio/Pitchfork-Reviews/blob/main/SentimentAnalysisClassifierGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!git clone https://github.com/ttb-folio/Pitchfork-Reviews.git

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW

     |████████████████████████████████| 2.1MB 10.6MB/s 
     |████████████████████████████████| 3.3MB 47.7MB/s 
     |████████████████████████████████| 901kB 24.0MB/s 
Cloning into 'Pitchfork-Reviews'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 155 (delta 37), reused 41 (delta 17), pack-reused 90
Receiving objects: 100% (155/155), 45.03 MiB | 26.34 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Checking out files: 100% (32/32), done.


In [2]:
#data = 'amazon'
data = 'pitchfork'

if data == "amazon":
    df = pd.read_csv('Pitchfork-Reviews/Data/amazon_cells_labelled.txt',  names=['review', 'sentiment'], sep='\t')
    reviews = df['review'].values
    labels = df['sentiment'].values
elif data == 'pitchfork':
    content_df = pd.read_csv('Pitchfork-Reviews/Data/content.csv')
    review_df = pd.read_csv('Pitchfork-Reviews/Data/reviews.csv')
    reviews = content_df.content.fillna(' ').values
    labels = (review_df.score <= 6).astype(int).values
    
reviews_train, reviews_test, y_train, y_test = train_test_split(reviews, labels, test_size=0.2, random_state=1000)

In [3]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize(x_text):
  text = list(x_text)
  tokenizedText = tokenizer(text, padding = True, truncation = True)
  return tokenizedText

x_train = tokenize(reviews_train)
x_test = tokenize(reviews_test)

In [4]:
class customDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx.tolist()
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

data_train = customDataset(x_train, y_train)
data_test = customDataset(x_test, y_test)

In [5]:
batch_size = 32
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

args = TrainingArguments(
    "./train_models",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.05,
    logging_dir = './logs',
    logging_steps = batch_size
)

trainer = Trainer(
    model,
    args,
    train_dataset = data_train,
    eval_dataset = data_test,
    tokenizer = tokenizer
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.396200,0.381325,30.760000,119.473000
2,0.297000,0.374561,30.760600,119.471000
3,0.251500,0.389152,30.765300,119.453000
4,0.196200,0.448878,30.760400,119.472000


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.396200,0.381325,30.760000,119.473000
2,0.297000,0.374561,30.760600,119.471000
3,0.251500,0.389152,30.765300,119.453000
4,0.196200,0.448878,30.760400,119.472000
5,0.155200,0.505378,30.754200,119.496000


TrainOutput(global_step=2300, training_loss=0.27035701109015425, metrics={'train_runtime': 2221.6749, 'train_samples_per_second': 1.035, 'total_flos': 1.51148203310592e+16, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 1875173376, 'init_mem_gpu_alloc_delta': 268953088, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 111755264, 'train_mem_gpu_alloc_delta': 816905728, 'train_mem_cpu_peaked_delta': 1867776, 'train_mem_gpu_peaked_delta': 13072364544})

In [7]:
predictions = trainer.predict(data_test)

In [8]:
from sklearn.metrics import confusion_matrix, f1_score

confusion_matrix(predictions[1],y_test)

array([[2979,    0],
       [   0,  696]])

In [ ]:
Ketchupredictions[2]

{'test_loss': 0.35213690996170044,
 'test_mem_cpu_alloc_delta': 0,
 'test_mem_cpu_peaked_delta': 0,
 'test_mem_gpu_alloc_delta': 0,
 'test_mem_gpu_peaked_delta': 79995904,
 'test_runtime': 0.2568,
 'test_samples_per_second': 778.836}

In [ ]:
predictions[1][20:30]

array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0])

In [ ]:
trainer.evaluate()

{'epoch': 10.0,
 'eval_loss': 0.35213690996170044,
 'eval_mem_cpu_alloc_delta': 4096,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 79992832,
 'eval_runtime': 0.1574,
 'eval_samples_per_second': 1270.961}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('train_models').to('cpu')
trainer = Trainer(
  model,
  TrainingArguments("poop"),
  train_dataset = data_train,
  eval_dataset = data_test,
  tokenizer = tokenizer
  )
outputs = model(torch.tensor(x_test['input_ids']))

'Baseline Sentiment Analysis.ipynb'   PitchforkReviewAnalysis.ipynb
 CurrentPF.ipynb		      Pitchfork-Reviews
 Data				      README.md
'Data Loading.ipynb'		      Reports
 EDA.ipynb			      runs
 logs				      train_models


In [ ]:
from transformers import get_linear_schedule_with_warmup
def fit(model, trainDataset, valDataset, epochs = 1, batchSize = 32, lr = 1e-3, scheduler=None):
  device = xm.xla_device()
  model = model.to(device)
  
  trainSampler = torch.utils.data.distributed.DistributedSampler(
    trainDataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)
  valSampler = torch.utils.data.distributed.DistributedSampler(
    valDataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=False)
  trainLoader = DataLoader(trainDataset, 
                           batch_size=batchSize, 
                           sampler=trainSampler, 
                           num_workers=0, 
                           drop_last = True)
  valLoader = DataLoader(valDataset, 
                         batch_size=batchSize, 
                         shuffle = False, 
                         sampler = valSampler,
                         num_workers=0,
                         drop_last=False)
  num_train_steps = int(len(trainDataset) / batchSize /xm.xrt_world_size()*epochs)
  optimizer = AdamW(model.parameters(), lr = lr)
  # scheduler = get_linear_schedule_with_warmup(
  #     optimizer, num_warmup_steps = 0, num_training_steps = num_train_steps
  # )
  # scheduler = torch.optim.lr_scheduler.StepLR(
  #     optimizer, step_size = 2, gamma = 0.1
  # )
  # scheduler = torch.optim.lr_scheduler.CyclicLR(
  #     optimizer,
  #     base_lr = 5e-5, max_lr = lr,
  #     mode = 'triangular2',
  #     step_size_up = int(len(trainDataset)/32/xm.xrt_world_size())*3, #3 being the number of epochs to go up.
  #     cycle_momentum = False
  # )
  def train_loop(trainLoader):
    optimizer.zero_grad()
  
    for batch in trainLoader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs[0]
      loss.backward()
      xm.optimizer_step(optimizer, barrier = True)
      if scheduler is not None:
        scheduler.step()
    print(f'Final Training Loss: {loss}')
  def eval_loop(valLoader):
    with torch.no_grad():
      for batch in valLoader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
      print(f'Final Val Loss: {loss}')

  for epoch in range(epochs):

    model.train()
    para_loader = pl.ParallelLoader(trainLoader,[device])
    train_loop(para_loader.per_device_loader(device))
    # _lr = scheduler.get_last_lr()[0]
    # print(_lr)
    print(f'epoch number: {epoch}')
    del para_loader
    
    model.eval()
    para_loader = pl.ParallelLoader(valLoader, [device])
    eval_loop(para_loader.per_device_loader(device))
    del para_loader
  xm.save(model.state_dict(), "model.pt")


In [ ]:
lr = 5e-4
def fit_multiprocessing(rank, flags):
  fit(model = DistilBertForSequenceClassification.from_pretrained(model_checkpoint,num_labels=2), 
      trainDataset = trainDataset, valDataset = valDataset, epochs = 20,  batchSize = 32, lr = lr)

FLAGS = {}
xmp.spawn(fit_multiprocessing, args =(FLAGS,), nprocs=8, start_method='fork')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Final Training Loss: 1.3492283821105957
epoch number: 0
Final Training Loss: 1.2990268468856812
Final Training Loss: 1.7547035217285156
Final Training Loss: 1.068077802658081
Final Training Loss: 1.7174874544143677
epoch number: 0
epoch number: 0
Final Training Loss: 1.5977767705917358
epoch number: 0
Final Training Loss: 1.4827221632003784
Final Training Loss: 1.4326668977737427
epoch number: 0
epoch number: 0
epoch number: 0
epoch number: 0
Final Val Loss: 1.555977463722229
Final Val Loss: 1.5560131072998047
Final Val Loss: 1.440492868423462
Final Val Loss: 1.2094310522079468
Final Val Loss: 2.0180861949920654
Final Val Loss: 1.440464735031128
Final Val Loss: 1.9026119709014893
Final Val Loss: 2.133604049682617
Final Training Loss: 0.7239044308662415
Final Training Loss: 0.7833437919616699
epoch number: 1
Final Training Loss: 0.6569998860359192
Final Training Loss: 0.7507330179214478
epoch number: 1
Final Training Loss: 0.6701981425285339
Final Training Loss: 0.7880423069000244
Final

In [ ]:
from transformers import DistilBertConfig

_model = DistilBertForSequenceClassification(
    config = DistilBertConfig(num_labels = 2))

checkpoint = torch.load('model.pt')
_model.load_state_dict(checkpoint)

_model.eval()

valLoader = DataLoader(valDataset, 
                        batch_size=32, 
                        shuffle = False, 
                        num_workers=0,
                        drop_last=True)
with torch.no_grad():
  for batch in valLoader:
    input_ids = batch['input_ids']

    outputs = _model(input_ids)
    break

  print(outputs[0])

tensor([[-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900],
        [-0.3900,  0.3900]])
